# Introduksjon til PySCF

**PySCF (Python-based Simulations of Chemistry Framework)** er en åpen kildekode-pakke for kvantekjemi som er utviklet for å utføre en ulike elektronstrukturteoriberegninger (langt ord!). Det brukes mye til både forskning og undervisning på grunn av sin fleksibilitet, effektivitet og brukervennlighet. Navnet er et ordspill – "SCF" står vanligvis for Self-Consistent Field, altså Hartree-Fock og Tetthetsfunksjonalteori (DFT). Disse metodene er blant PySCFs sterkeste sider.

Dokumentasjonen er god, og et godt sted å starte er den offisielle nettsiden: https://pyscf.org/index.html


### Noen nøkkelfunksjoner i PySCF:
1. **Kjernefunksjonaliteter**:
   - PySCF støtter **Hartree-Fock (HF)**- og **Tetthetsfunksjonalteori (DFT)**-beregninger, samt avanserte metoder etter Hartree-Fock som **Møller-Plesset pertubasjonsteori (MP2)**, **Coupled Cluster (CC)**, **Konfigurasjonsinteraksjon (CI)**, **Complete Active Space Self-Consistent Field (CASSCF)**-beregninger og flere.

2. **Basis-sett og integraler**:
   - Programvaren inkluderer en rekke **Gaussiske orbitaler (GTO)** basis-sett og effektive rutiner for å beregne integraler.

3. **Modularitet og fleksibilitet**:
   - PySCF er designet for å være modulær, noe som gjør det mulig for brukere å utvide funksjonaliteten eller integrere den med andre verktøy. Dette gjør den egnet for skreddersydde arbeidsflyter i forskning.

4. **Python-grensesnitt**:
   - Skrevet i Python med ytelseskritiske deler i C, tilbyr PySCF et intuitivt og brukervennlig grensesnitt som utnytter Python-økosystemet for skripting og rask utvikling.
  

# En første beregning med Hartree-Fock

I denne delen av notebooken skal vi se hvordan vi definerer et molekyl, velger et GTO-basissett, og kjører en Hartree-Fock-beregning på molekylet.

## Importering av relevante pakker

In [1]:
from pyscf import * 
import matplotlib.pyplot as plt

## Definere et molekyl

Et molekyl defineres ved å plassere kjerner på posisjoner i rommet gitt ved XYZ-koordinater. PySCF regner ut hvor mange elektroner molekylet har under antagelsen at det er elektrisk nøytralt. Her definerer vi et vannmolekyl (med en litt rar geometri), og - for moro skyld - koffein-molekylet (bør være nær likevekt).

Når geometrien er bestemt, må man så velge et AO basissett. Husk at AO står for "atomic orbital". Her velger vi et basissett som heter STO-3G. Dette er en såkalt "minimal basis", der hvert elektron starter med 1 orbital, sentrert på atomet elektronet "hører til". I dette eksempelet vil det derfor være 1 basisfunksjon for hvert H-atom, og 5 basisfunksjoner for O-atomet. Hver basisfunksjon er en Slater-type orbital som er tilnærmet med 3 gaussiske orbitaler.

Så må vi be PySCF å "bygge" molekylet for å få en veldefinert datastruktur som PySCF kan gjøre beregninger med.

For mer informasjon og flere måter å gjøre dette på, se https://pyscf.org/user/gto.html.



In [74]:
water = """
O          0.00000        0.00000        0.00000
H          0.27740        0.89290        0.25440
H          0.60680       -0.23830       -0.71690

"""

caffeine = """
O          0.47000        2.56880        0.00060
O         -3.12710       -0.44360       -0.00030
N         -0.96860       -1.31250        0.00000
N          2.21820        0.14120       -0.00030
N         -1.34770        1.07970       -0.00010
N          1.41190       -1.93720        0.00020
C          0.85790        0.25920       -0.00080
C          0.38970       -1.02640       -0.00040
C          0.03070        1.42200       -0.00060
C         -1.90610       -0.24950       -0.00040
C          2.50320       -1.19980        0.00030
C         -1.42760       -2.69600        0.00080
C          3.19260        1.20610        0.00030
C         -2.29690        2.18810        0.00070
H          3.51630       -1.57870        0.00080
H         -1.04510       -3.19730       -0.89370
H         -2.51860       -2.75960        0.00110
H         -1.04470       -3.19630        0.89570
H          4.19920        0.78010        0.00020
H          3.04680        1.80920       -0.89920
H          3.04660        1.80830        0.90040
H         -1.80870        3.16510       -0.00030
H         -2.93220        2.10270        0.88810
H         -2.93460        2.10210       -0.88490
"""


mol = gto.Mole()
mol.atom = water
mol.basis = 'cc-pVDZ'
mol.unit = 'Angstrom' # Alternativt 'Bohr'
mol.build()



## Kjøre en Hartree-Fock-beregning

Når vi har bestemt molekylet vårt, kan vi gjøre, for eksempel, en HF-beregning. Dette gjøres i to steg:
1. Lage et `RHF`-objekt som lar oss gjøre en Restricted Hartree-Fock-beregning. Dette objektet initialiseres med `Mole`-objektet fra tidligere.
2. Gjøre beregningene.



In [75]:
mf = scf.RHF(mol)  # Set up a Restricted Hartree-Fock solver
energy = mf.kernel()   # Solve for the energy

print("Energy: ", energy)

converged SCF energy = -76.0259894063641
Energy:  -76.0259894063641


## Valg av basissett

Kvantekjemi inneholder et veritabelt overflødighetshorn av GTO-baserte basissett, tilpasset mange ulike formål. I eksempelet over ble basissettet `STO-3G` brukt. Her er en tabell med flere basissett å prøve ut:

| **Basissett**    | **Type**        | **Beskrivelse**                                                                                                      | **Typiske bruksområder**                   |
|------------------|-----------------|----------------------------------------------------------------------------------------------------------------------|--------------------------------------------|
| **STO-3G**       | Minimalt Basis   | Bruker 3 Gauss-funksjoner for å tilnærme én Slater-type orbital. Rask, men med minimal nøyaktighet.                   | Små molekyler, enkle beregninger           |
| **3-21G**        | Split-Valence    | Splitter valensorbitalene i to Gauss-funksjoner for større fleksibilitet.                                             | Små systemer, moderat nøyaktighet          |
| **6-31G**        | Split-Valence    | Mer nøyaktig enn 3-21G; bruker 6 Gauss-funksjoner for kjernorbitaler og en delt valensorbitaltilnærming.              | Organiske molekyler, geometrioptimalisering|
| **cc-pVDZ**      | Korrelasjons-Konsistent | Dobbelt-zeta basissett designet for korrelerte bølgefunksjonsmetoder som CCSD og MP2.                               | Korrelerte beregninger, små molekyler      |
| **cc-pVTZ**      | Korrelasjons-Konsistent | Trippel-zeta versjon for bedre nøyaktighet, spesielt for korrelerte metoder.                                         | Nøyaktige korrelerte metoder               |
| **cc-pVQZ**      | Korrelasjons-Konsistent | Kvadrupel-zeta versjon for svært høy nøyaktighet i korrelerte beregninger.                                           | Høynøyaktige beregninger, referanseverdier |
| **def2-SVP**     | Split-Valence    | Dobbelt-zeta med polarisasjonsfunksjoner, optimalisert for DFT og korrelerte metoder.                                | Middels store molekyler, geometrioptimalisering |
| **def2-TZVP**    | Trippel-Zeta     | Trippel-zeta basissett med polarisasjon. Høy nøyaktighet, egnet for større systemer.                                  | DFT og bølgefunksjonsbaserte metoder       |
| **def2-QZVP**    | Kvadrupel-Zeta   | Kvadrupel-zeta basissett for ekstremt høy nøyaktighet, spesielt i post-HF metoder.                                    | Høynøyaktige beregninger                   |
| **LANL2DZ**      | ECP              | Bruker effektive kjernpotensialer (ECP) for tunge atomer, med dobbelt-zeta basissett for valenselektroner.            | Tunge elementer, overgangsmetaller         |
| **AUG-cc-pVDZ**  | Augmentert Basis | cc-pVDZ med diffuse funksjoner for å fange opp langdistanse-interaksjoner, bedre behandling av svakt bundne systemer. | Svake interaksjoner, anioner, eksiterte tilstander |
| **AUG-cc-pVTZ**  | Augmentert Basis | cc-pVTZ med diffuse funksjoner for mer nøyaktig behandling av eksiterte tilstander og anioner.                        | Svakt bundne systemer, van der Waals interaksjoner |


# Regne ut egenskaper til molekylet

Etter at HF-beregningen er ferdig, kan vi be PySCF om å regne ut ulike egenskaper. Her viser vi hvordan man regner ut dipolmomentet og Mulliken-populasjoner.


In [76]:
dip = mf.dip_moment()
print("Dipole moment: ", dip)
pop = mf.mulliken_pop()
print("Mulliken population: ", pop)



Dipole moment(X, Y, Z, Debye):  1.53781,  1.13848, -0.80438
Dipole moment:  [ 1.53780764  1.13848495 -0.80438399]
 ** Mulliken pop  **
pop of  0 O 1s            2.00171
pop of  0 O 2s            0.83481
pop of  0 O 3s            0.83445
pop of  0 O 2px           0.99446
pop of  0 O 2py           0.86620
pop of  0 O 2pz           0.94974
pop of  0 O 3px           0.68638
pop of  0 O 3py           0.52066
pop of  0 O 3pz           0.61789
pop of  0 O 3dxy          0.00246
pop of  0 O 3dyz          0.00028
pop of  0 O 3dz^2         0.00193
pop of  0 O 3dxz          0.00403
pop of  0 O 3dx2-y2       0.00328
pop of  1 H 1s            0.68255
pop of  1 H 2s            0.07353
pop of  1 H 2px           0.02023
pop of  1 H 2py           0.04303
pop of  1 H 2pz           0.02152
pop of  2 H 1s            0.68255
pop of  2 H 2s            0.07353
pop of  2 H 2px           0.02926
pop of  2 H 2py           0.02009
pop of  2 H 2pz           0.03544
 ** Mulliken atomic charges  **
charge of    0O =

# Gjøre en DFT-beregning

Nå viser vi hvordan man kan gjøre en Kohn-Sham DFT-beregning i PySCF. Dette er ganske likt som Hartree-Fock-beregningen. Vi regner på koffein-molekylet.

Her er en tabell med XC-funksjonaler du kan prøve:

| **Funksjonal**            | **Definisjonsstreng** | **Beskrivelse**                                                                                          |
|---------------------------|-----------------------|----------------------------------------------------------------------------------------------------------|
| **LDA (Local Density Approximation)**        | `'lda'`               | Lokal tetthets-tilnærming. XC-funksjonalen avhenger bare av elektron-tettheten i hvert punkt.               |
| **PBE (Perdew-Burke-Ernzerhof)**  | `'pbe'`               | Generalisert gradient-tilnærming (GGA). Inkluderer gradienten av tettheten for mer nøyaktige beregninger.   |
| **BLYP (Becke-Lee-Yang-Parr)**     | `'blyp'`              | Kombinasjon av Becke's utvekslingsfunksjonal og LYP's korrelasjonsfunksjonal. GGA-nivå funksjonal.           |
| **BP86**                   | `'bp86'`              | Kombinasjon av Becke’s 1988 utvekslingsfunksjonal og Perdew's 1986 korrelasjonsfunksjonal.                  |
| **B3LYP**                  | `'b3lyp'`             | Hybrid GGA funksjonal som kombinerer BLYP med en andel av Hartree-Fock utvekslingsenergi.                   |
| **HSE (Heyd-Scuseria-Ernzerhof)**  | `'hse'`               | Hybrid GGA funksjonal med skjermet utveksling for forbedret behandling av systemer med lange rekkevidde.     |
| **M06**                    | `'m06'`               | Hybrid meta-GGA funksjonal, som inkluderer tetthetsgradienter og kinetisk energitetthet.                    |
| **TPSS (Tao-Perdew-Staroverov-Scuseria)** | `'tpss'`              | Meta-GGA funksjonal som tar hensyn til både elektron-tetthet og kinetisk energitetthet.                     |
| **SCAN**                   | `'scan'`              | Meta-GGA funksjonal som bruker strengt korrelerte tettheter for høyere nøyaktighet i komplekse systemer.    |
| **CAM-B3LYP**              | `'cam-b3lyp'`         | Hybrid funksjonal med lang-rekkevidde korreksjon, basert på B3LYP, men med tilleggsparameter for screening. |
| **PBE0**                   | `'pbe0'`              | Hybridversjon av PBE, som inkluderer en andel av Hartree-Fock utvekslingsenergi.                            |


In [77]:

mf_dft = scf.RKS(mol)

mf_dft.xc = 'b3lyp' # Velg en DFT-funksjonal

energy = mf_dft.kernel()
print("Energy: ", energy)

converged SCF energy = -76.4205861688998
Energy:  -76.42058616889985


# Geometrioptimering

Her viser vi hvordan vi kan gjøre en geometrioptimering. Da prøver PySCF å finne den geometrien som gir lavest energy med modellen og basissettet vi har valgt.


In [78]:
from pyscf.geomopt.geometric_solver import optimize
mol_eq = optimize(mf, maxsteps=100)
print(mol_eq.tostring())


                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********      .))))))  
          ##      .%%%%%%/      (%%%%%%,                  ,******      /)))))  
        %%%%%%      .%%%%%%#      *%%%%%%,    ,/////.       ******      )))))) 
      #%      %%      .%%%%%%/      *%%


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.277400   0.892900   0.254400    0.000000  0.000000  0.000000
   H   0.606800  -0.238300  -0.716900    0.000000  0.000000  0.000000
converged SCF energy = -76.0259894063671
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0217754720    -0.0161216894     0.0113897562
1 H     0.0067301756     0.0223392879     0.0065650038
2 H     0.0150452964    -0.0062175985    -0.0179547600
----------------------------------------------
cycle 1: E = -76.0259894064  dE = -76.026  norm(grad) = 0.0451513


Step    0 : Gradient = 2.607e-02/2.939e-02 (rms/max) Energy = -76.0259894064
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 1.60000e-01 5.35840e-01 5.35841e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.006670   0.004938  -0.003488    0.006670  0.004938 -0.003488
   H   0.278342   0.875744   0.243534    0.000942 -0.017156 -0.010866
   H   0.599189  -0.226082  -0.702545   -0.007611  0.012218  0.014355

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -76.0270293136351
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000829013    -0.0000613006     0.0000434062
1 H     0.0005675888    -0.0017762706    -0.0015731807
2 H    -0.0004846875     0.0018375711     0.0015297746
----------------------------------------------
cycle 2: E = -76.0270293136  dE = -0.00103991  norm(grad) = 0.00345203


Step    1 : Displace = 1.739e-02/2.033e-02 (rms/max) Trust = 1.000e-01 (=) Grad = 1.993e-03/2.440e-03 (rms/max) E (change) = -76.0270293136 (-1.040e-03) Quality = 0.948
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 1.59996e-01 5.35840e-01 5.79736e-01



Geometry optimization cycle 3
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.009214   0.006822  -0.004820    0.002545  0.001884 -0.001331
   H   0.275853   0.878980   0.247787   -0.002489  0.003236  0.004253
   H   0.599133  -0.231202  -0.705467   -0.000056 -0.005120 -0.002922

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -76.0270532769522
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0000855380     0.0000633299    -0.0000447404
1 H    -0.0000834510     0.0001080369     0.0001423260
2 H    -0.0000020870    -0.0001713669    -0.0000975856
----------------------------------------------
cycle 3: E = -76.027053277  dE = -2.39633e-05  norm(grad) = 0.000301854


Step    2 : Displace = 5.206e-03/5.895e-03 (rms/max) Trust = 1.414e-01 (+) Grad = 1.743e-04/1.972e-04 (rms/max) E (change) = -76.0270532770 (-2.396e-05) Quality = 0.900
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 1.73711e-01 5.35840e-01 5.85492e-01



Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.008942   0.006621  -0.004677   -0.000272 -0.000201  0.000142
   H   0.276095   0.878718   0.247404    0.000242 -0.000262 -0.000383
   H   0.599163  -0.230738  -0.705226    0.000030  0.000464  0.000241
converged SCF energy = -76.0270535126751
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000068424    -0.0000050675     0.0000035780
1 H     0.0000021321     0.0000069631     0.0000020136
2 H     0.0000047103    -0.0000018956    -0.0000055915
----------------------------------------------
cycle 4: E = -76.0270535127  dE = -2.35723e-07  norm(grad) = 1.4122e-05


Step    3 : Displace = 4.769e-04/5.233e-04 (rms/max) Trust = 2.000e-01 (+) Grad = 8.153e-06/9.236e-06 (rms/max) E (change) = -76.0270535127 (-2.357e-07) Quality = 1.004
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    
Time elapsed since start of run_optimizer: 0.210 seconds


O           0.00894        0.00662       -0.00468
H           0.27609        0.87872        0.24740
H           0.59916       -0.23074       -0.70523


# Beregne vibrasjonelle frekvenser

Som eksempel på hva PySCF kan gjøre viser vi hvordan man beregner vibrasjonelle frekvenser.

Her bruker vi den optimerte geometrien fra forrige beregning. Merk at modellen vår ikke er veldig nøyaktig, så de vibrasjonelle frekvensene er heller ikke veldig nøyaktige.


In [85]:
mf.mol = mol_eq
mf.kernel()

from pyscf import eph
myeph = eph.EPH(mf)

mat, omega = myeph.kernel()

au_to_cmm1 = 219474.63 
print('Vibrasjonelle frekvenser i invers cm:')
print(omega * au_to_cmm1)

converged SCF energy = -76.5831092005586
Vibrasjonelle frekvenser i invers cm:
[4950.77292193 4949.76015614 2015.66750482]
